In [1]:
import os
import findspark
import setuptools
import datetime
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType, LongType, IntegerType
from pyspark.sql import functions as f
from pyspark.sql.functions import col
from functools import reduce

In [2]:
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-22.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "/Users/adilsonyamaguchi/Apps/spark"

In [3]:
findspark.init()

In [4]:
spark = SparkSession.builder \
    .master('local[2]') \
    .appName("ProcessarParquet") \
    .config("spark.ui.port", '4050') \
    .getOrCreate()

24/07/23 22:52:16 WARN Utils: Your hostname, MacBook-Pro-de-Adilson.local resolves to a loopback address: 127.0.0.1; using 192.168.15.102 instead (on interface en0)
24/07/23 22:52:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/23 22:52:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

24/07/23 22:52:29 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [23]:
eTipo_PRODUCAO = {"codigo": 2, "nome": "Producao", "subtipo":None}
eTipo_PROCESSAMENTO = {"codigo": 3, "nome":"Processamento", "subtipo":[
    {"codigo":1, "nome":"Viníferas"}
    , {"codigo":2, "nome":"Americanas e híbridas"}
    , {"codigo":3, "nome":"Uvas de mesa"}
    , {"codigo":4, "nome":"Sem classificação"}
    ]}
eTipo_COMERCIALIZACAO = {"codigo": 4, "nome":"Comercializacao", "subtipo":None}
eTipo_IMPORTACAO = {"codigo":5, "nome":"Importacao", "subtipo":[
        {"codigo":1, "nome":"Vinhos de mesa"}
        , {"codigo":2, "nome":"Espumantes"}
        , {"codigo":3, "nome":"Uvas frescas"}
        , {"codigo":4, "nome":"Uvas passas"}
        , {"codigo":5, "nome":"Suco de uva"}
        ]}
eTipo_EXPORTACAO = {"codigo":6, "nome":"Exportacao", "subtipo":[
    {"codigo":1, "nome":"Vinhos de mesa"}
    , {"codigo":2, "nome":"Espumantes"}
    , {"codigo":3, "nome":"Uvas frescas"}
    , {"codigo":4, "nome":"Suco de uva"}
    ]}

In [25]:
tipo = eTipo_PROCESSAMENTO
print(tipo["nome"])

Processamento


In [27]:
from selenium import webdriver # importa módulo webdriver para realizar webscraping
from selenium.webdriver.chrome.options import Options # importa módulo Options para definição de parametros para a execução do browser chrome
from fastapi import HTTPException # importa módulo para o tratamento de exeções  HTTP
import json # importa módulo para tratamento de dados JSON
from selenium.webdriver.common.by import By # importa módulo By que traz alguns parãmetros para navegação durante o scraping
import time # importa o módulo time para ajudar a trabalhar com a tempo
from datetime import datetime # importa o módulo datetime para ajudar a trabalhar com datas
import pandas as pd # importa módulo pandas para trabalhar com dados de maneira simples
from lxml import etree # importa lxlm para trabalhar com conteúdo HTML
from io import StringIO # importa a biblioteca io para trabalhar com o sistema de arquivos
import pyarrow
import os
import sys

In [135]:
#tipo = eTipo_PRODUCAO
tipo = eTipo_COMERCIALIZACAO
anoInicio = 1970
anoTermino = 2024

dataframes_ano = []

for ano in range(anoInicio, anoTermino+1):

    file_name_html = f"../data/html/{tipo["nome"]}_{ano}.html"

    with open(file_name_html, mode="r", encoding='utf-8') as html:
        html_content = html.read()
        parser = etree.HTMLParser()
        tree = etree.HTML(html_content, parser)
        table_elements = tree.xpath('//table[contains(@class, "tb_base") and contains(@class, "tb_dados")]')

        # Lista para armazenar os DataFrames
        dfs = []

        for i, table in enumerate(table_elements):
            # Obter o HTML da tabela
            table_html = etree.tostring(table, pretty_print=True, encoding='unicode')
            
            # Converter o HTML da tabela em DataFrame usando pandas
            df = pd.read_html(StringIO(table_html))[0]
            df["ano"] = ano
            df["tipo"] = tipo["nome"]
            dfs.append(df)
        
    dataframes_ano.append(pd.concat(dfs, ignore_index=True))

df = pd.concat(dataframes_ano, ignore_index=True)
     
arquivo_parquet = f"../data/parquet/{tipo['nome']}_{anoInicio}{f'_{anoTermino}' if anoTermino is not None and anoTermino != anoInicio else ''}.parquet"
df.to_parquet(arquivo_parquet, engine='pyarrow')

In [116]:
df['ano'].unique()

array([1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980,
       1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

In [138]:
#tipo = eTipo_PROCESSAMENTO
#tipo = eTipo_IMPORTACAO
tipo = eTipo_EXPORTACAO
anoInicio = 1970
anoTermino = 2024


listadf=[]
dataframes_tipo = []

for subtipo in tipo['subtipo']:
    print(f"codigo={subtipo['codigo']} nome={subtipo['nome']}")

    dataframes_ano = []
    for ano in range(anoInicio, anoTermino+1):

        file_name_html = f"../data/html/{tipo["nome"]}_{subtipo['nome']}_{ano}.html"

        with open(file_name_html, mode="r", encoding='utf-8') as html:
            html_content = html.read()
            parser = etree.HTMLParser()
            tree = etree.HTML(html_content, parser)
            table_elements = tree.xpath('//table[contains(@class, "tb_base") and contains(@class, "tb_dados")]')

            # Lista para armazenar os DataFrames
            dfs = []

            for i, table in enumerate(table_elements):
                # Obter o HTML da tabela
                table_html = etree.tostring(table, pretty_print=True, encoding='unicode')
                
                # Converter o HTML da tabela em DataFrame usando pandas
                df = pd.read_html(StringIO(table_html))[0]
                df["ano"] = ano
                df["tipo"] = subtipo['nome']
                dfs.append(df)
            
        dataframes_ano.append(pd.concat(dfs, ignore_index=True))
        
    dataframes_tipo = pd.concat(dataframes_ano)
    arquivo_parquet = f"../data/parquet/{tipo['nome']}_{subtipo['nome']}_{anoInicio}{f'_{anoTermino}' if anoTermino is not None and anoTermino != anoInicio else ''}.parquet"
    df.to_parquet(arquivo_parquet, engine='pyarrow')

codigo=1 nome=Vinhos de mesa
codigo=2 nome=Espumantes
codigo=3 nome=Uvas frescas
codigo=4 nome=Suco de uva


In [132]:
listadf[4]['tipo'].unique()

IndexError: list index out of range

In [59]:
df_70

,Cultivar,Quantidade (Kg),ano,Sem definição
0,TINTAS,10.448.228,1970,NaN
1,Alicante Bouschet,-,1970,NaN
2,Ancelota,-,1970,NaN
3,Aramon,-,1970,NaN
4,Alfrocheiro,-,1970,NaN
...,...,...,...,...
8,Italia,3.733,1970,NaN
9,"Rubi (Itália, Itália Ro)",-,1970,NaN
10,Total,60.876,1970,NaN
0,NaN,3.675.463,1970,Sem classificação
